In [1]:
#IMPORTS

import pandas as pd
import statsmodels.api as sm
from statsmodels.miscmodels.ordinal_model import OrderedModel
from statsmodels.stats.multitest import fdrcorrection
from scipy import stats
import numpy as np

In [2]:
full_analysis_df = pd.read_excel('c:/Users/Hailey Sherman/ClimateOfHopeData/Data_2025/COMBINED_TEXT_jan-june_pre_in-gallery_post_w_SASSY_urban_rural.xlsx')
#full_analysis_df = pd.read_excel('c:/Users/u1172791/COH files/Data_2025/COMBINED_TEXT_jan-june_pre_in-gallery_post_w_SASSY_urban_rural.xlsx')
display(full_analysis_df)

,Unnamed: 0,start_date,end_date,status,ip_address,progress,duration_in_seconds,finished,recorded_date,response_id,...,q49_48,q49_49,q49_50,q51_1,q51_2,q51_3,q51_4,q51_5,q51_6,q51_7
0,0,2024-04-05 14:12:54,2024-04-05 15:57:31,IP Address,155.98.131.1,100.0,6276.0,1.0,2024-04-05 15:57:31,R_2dPUXEtypfBvZhD,...,A little bit,Not at all,Not at all,A lot,A lot,A lot,A lot,A little bit,Moderately,A lot
1,1,2024-03-13 18:59:27,2024-03-13 20:15:18,IP Address,155.98.131.5,100.0,4550.0,1.0,2024-03-13 20:15:19,R_7PAooFY6nTwIqA7,...,A little bit,Not at all,Not at all,A great deal,A great deal,A great deal,Moderately,Not at all,Moderately,A lot
2,2,2024-03-23 13:49:19,2024-04-03 19:27:57,IP Address,155.98.131.5,100.0,970717.0,1.0,2024-04-03 19:27:58,R_73AteLV2ZE5L4GJ,...,A lot,Moderately,A lot,Moderately,A little bit,A little bit,A little bit,A little bit,Moderately,A little bit
3,3,2024-01-26 15:51:59,2024-01-26 16:41:06,IP Address,155.98.131.2,100.0,2946.0,1.0,2024-01-26 16:41:06,R_3uP2DqVWkGDu8La,...,Extremely,Moderately,Not at all,Moderately,A lot,A great deal,Moderately,A little bit,Moderately,Moderately
4,4,2024-01-24 16:32:42,2024-01-24 19:27:07,IP Address,155.98.131.2,100.0,10464.0,1.0,2024-01-24 19:27:07,R_7OiAhfM2TEkyQdM,...,A lot,Moderately,Not at all,Not at all,Not at all,Not at all,Not at all,A little bit,Not at all,Not at all
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186,186,2024-04-06 14:34:46,2024-04-06 15:31:05,IP Address,155.98.131.5,100.0,3378.0,1.0,2024-04-06 15:31:05,R_3fNzlkOukIhRV9V,...,Not at all,Moderately,Not at all,A great deal,A great deal,A great deal,A great deal,A little bit,A great deal,A great deal
187,187,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
188,188,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
189,189,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# get emotion columns and set order of them
# Define the prefixes we're looking for
emotion_prefixes = ['aspen_', 'here_now_impacts_', 'build_momentum_', 'connecting_']

# Get all columns that start with any of these prefixes
emotion_columns = [col for col in full_analysis_df.columns 
                  if any(col.startswith(prefix) for prefix in emotion_prefixes)]

# Convert emotion columns to ordinal type, and define the order of the categories
for col in emotion_columns:
    full_analysis_df[col] = pd.Categorical(
        full_analysis_df[col],
        categories=['Not at all', 'Only a little', 'A moderate amount', 'A great deal'],
        ordered=True
    )

In [5]:
#define Sassy order
full_analysis_df['SASSY'] = pd.Categorical(
        full_analysis_df['SASSY'],
        categories=['Dismissive', 'Doubtful', 'Cautious', 'Concerned', 'Alarmed'],
        ordered=True
    )

In [6]:
#run preliminary tests on age

# 1. Shapiro-Wilk test for normality
shapiro_stat, shapiro_p = stats.shapiro(full_analysis_df['age'].dropna())

# 2. Grubbs test for outliers
def grubbs_test(data):
    mean = data.mean()
    std = data.std()
    n = len(data)
    # Calculate test statistics for both max and min values
    G_max = (data.max() - mean) / std
    G_min = (mean - data.min()) / std
    # Critical value (approximate for alpha = 0.05)
    critical_value = (n - 1) / np.sqrt(n) * np.sqrt(
        stats.t.ppf(1 - 0.05 / (2*n), n-2)**2 / 
        (n - 2 + stats.t.ppf(1 - 0.05 / (2*n), n-2)**2)
    )
    return G_max > critical_value or G_min > critical_value

# 3. Check skewness and kurtosis
skewness = stats.skew(full_analysis_df['age'].dropna())
skewness_test = stats.skewtest(full_analysis_df['age'].dropna())
kurtosis = stats.kurtosis(full_analysis_df['age'].dropna())
kurtosis_test = stats.kurtosistest(full_analysis_df['age'].dropna())

print("Distribution Tests for Age:")
print("-" * 50)
print(f"Shapiro-Wilk test (normality):")
print(f"  Statistic: {shapiro_stat:.4f}")
print(f"  p-value: {shapiro_p:.4f}")
print(f"  Interpretation: {'Non-normal' if shapiro_p < 0.05 else 'Normal'} distribution")
print("\nOutlier Test (Grubbs):")
print(f"  Has outliers: {grubbs_test(full_analysis_df['age'].dropna())}")
print("\nSkewness:")
print(f"  Value: {skewness:.4f}")
print(f"  Test statistic: {skewness_test.statistic:.4f}")
print(f"  p-value: {skewness_test.pvalue:.4f}")
print(f"  Interpretation: {'Significantly skewed' if skewness_test.pvalue < 0.05 else 'Not significantly skewed'}")
print("\nKurtosis:")
print(f"  Value: {kurtosis:.4f}")
print(f"  Test statistic: {kurtosis_test.statistic:.4f}")
print(f"  p-value: {kurtosis_test.pvalue:.4f}")
print(f"  Interpretation: {'Significant kurtosis' if kurtosis_test.pvalue < 0.05 else 'Normal kurtosis'}")

Distribution Tests for Age:
--------------------------------------------------
Shapiro-Wilk test (normality):
  Statistic: 0.9078
  p-value: 0.0000
  Interpretation: Non-normal distribution

Outlier Test (Grubbs):
  Has outliers: False

Skewness:
  Value: 0.9394
  Test statistic: 4.6992
  p-value: 0.0000
  Interpretation: Significantly skewed

Kurtosis:
  Value: 0.0375
  Test statistic: 0.3553
  p-value: 0.7223
  Interpretation: Normal kurtosis


In [7]:
#run ordinal logistic regression on age vs emotions in each exhibit section
# First, let's ensure age is numeric and handle any missing values
full_analysis_df['age'] = pd.to_numeric(full_analysis_df['age'], errors='coerce')

# We'll create a function to run ordinal regression for each emotion column
def run_ordinal_regression(data, dependent_var, independent_var='age'):
     # Create a copy of the data with only the columns we need
    model_data = data[[dependent_var, independent_var]].copy()
    
    # Drop rows with missing values
    model_data = model_data.dropna()
    
    # Convert categorical to numeric while preserving order
    y = model_data[dependent_var].cat.codes
    X = model_data[[independent_var]]
    
    # Fit ordinal logistic regression
    model = OrderedModel(y, X, distr='logit')
    results = model.fit()
    
    return results, len(model_data)  # Return both results and sample size

# To run for all emotion columns
emotion_results = {}
all_results_stats = {}  # New dictionary to store all results
significant_results = {}  # Dictionary for significant results

for emotion in emotion_columns:
    try:
        results, n = run_ordinal_regression(full_analysis_df, emotion)
        emotion_results[emotion] = results

        # Store results in our summary dictionaries
        age_coef = results.params['age']
        age_pval = results.pvalues['age']
        
        # Store in all_results
        all_results_stats[emotion] = {
            'coefficient': age_coef,
            'p_value': age_pval,
            'sample_size': n
        }
        
        # If significant, store in significant_results
        if age_pval < 0.05:
            significant_results[emotion] = {
                'coefficient': age_coef,
                'p_value': age_pval,
                'sample_size': n
            }

        print(f"\nResults for {emotion} (n={n}):")
        print(results.summary().tables[1])  # Print just the coefficients table for brevity
    except Exception as e:
        print(f"Error processing {emotion}: {str(e)}")



Optimization terminated successfully.
         Current function value: 1.066088
         Iterations: 96
         Function evaluations: 166

Results for aspen_indifferent (n=184):
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
age           -0.0154      0.010     -1.536      0.124      -0.035       0.004
0/1           -0.2436      0.345     -0.707      0.480      -0.919       0.432
1/2            0.2871      0.131      2.187      0.029       0.030       0.544
2/3            0.5179      0.208      2.491      0.013       0.110       0.925
Optimization terminated successfully.
         Current function value: 1.168568
         Iterations: 114
         Function evaluations: 198

Results for aspen_sad (n=184):
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
age           -0.

In [8]:
# print summary of significant findings for age
print("\n" + "="*50)
print("\nSignificant relationships between age and emotions (p < 0.05):")
print("\nEmotion | Coefficient | P-value | n")
print("-" * 50)
for emotion, stats in significant_results.items():
    print(f"{emotion} | {stats['coefficient']:.4f} | {stats['p_value']:.4f} | {stats['sample_size']}")

if not significant_results:
    print("No significant relationships found between age and emotions.")



Significant relationships between age and emotions (p < 0.05):

Emotion | Coefficient | P-value | n
--------------------------------------------------
aspen_stressed | -0.0228 | 0.0345 | 184
build_momentum_interested | 0.0237 | 0.0139 | 184
connecting_angry_frustrated | 0.0245 | 0.0167 | 184


In [9]:
#Calculate Bonferonni alpha level
# Count how many tests we're running
n_tests = len(emotion_columns)
print(f"Number of tests: {n_tests}")

# Adjust our alpha level (0.05/number of tests)
bonferroni_alpha = 0.05 / n_tests
print(f"\nBonferroni-corrected alpha level: {bonferroni_alpha}")

Number of tests: 52

Bonferroni-corrected alpha level: 0.0009615384615384616


In [10]:
# Run FDR correction

# First, collect all p-values and corresponding information
all_pvals = []
emotion_order = []
coefficients = []
sample_sizes = []

for emotion in emotion_columns:
    try:
        results, n = run_ordinal_regression(full_analysis_df, emotion)
        all_pvals.append(results.pvalues['age'])
        emotion_order.append(emotion)
        coefficients.append(results.params['age'])
        sample_sizes.append(n)
    except Exception as e:
        print(f"Error processing {emotion}: {str(e)}")

# Apply FDR correction
rejected, adjusted_pvals = fdrcorrection(all_pvals, alpha=0.05, method='indep')

# Print all results, sorted by original p-value
print("\nAll results sorted by p-value:")
print("\nEmotion | Coefficient | Original P | Adjusted P | Significant after FDR")
print("-" * 85)

# Create sorted indices
sorted_indices = sorted(range(len(all_pvals)), key=lambda k: all_pvals[k])

for i in sorted_indices:
    print(f"{emotion_order[i]} | {coefficients[i]:.4f} | {all_pvals[i]:.4f} | {adjusted_pvals[i]:.4f} | {'Yes' if rejected[i] else 'No'}")

# Print summary of significant results
print("\n" + "="*50)
print("\nSignificant relationships after FDR correction (q < 0.05):")
print("\nEmotion | Coefficient | Original P | Adjusted P | n")
print("-" * 70)
significant_count = 0
for i in sorted_indices:
    if rejected[i]:
        significant_count += 1
        print(f"{emotion_order[i]} | {coefficients[i]:.4f} | {all_pvals[i]:.4f} | {adjusted_pvals[i]:.4f} | {sample_sizes[i]}")

if significant_count == 0:
    print("No relationships remained significant after FDR correction.")

print(f"\nNumber of tests conducted: {len(all_pvals)}")
print(f"Number of significant results before correction: {sum(p < 0.05 for p in all_pvals)}")
print(f"Number of significant results after FDR correction: {sum(rejected)}")

Optimization terminated successfully.
         Current function value: 1.066088
         Iterations: 96
         Function evaluations: 166


Optimization terminated successfully.
         Current function value: 1.168568
         Iterations: 114
         Function evaluations: 198
Optimization terminated successfully.
         Current function value: 1.085470
         Iterations: 106
         Function evaluations: 183
Optimization terminated successfully.
         Current function value: 1.144117
         Iterations: 91
         Function evaluations: 159
Optimization terminated successfully.
         Current function value: 1.060101
         Iterations: 114
         Function evaluations: 193
Optimization terminated successfully.
         Current function value: 1.280870
         Iterations: 60
         Function evaluations: 115
Optimization terminated successfully.
         Current function value: 1.162620
         Iterations: 106
         Function evaluations: 177
Optimization terminated successfully.
         Current function value: 1.358552
         Iterations: 86
         Function evaluations: 151
Optimization terminated

In [ ]:
# Filter for just Male/Female responses and run gender analysis
gender_df = full_analysis_df[full_analysis_df['gender'].isin(['Male', 'Female'])].copy()

# Convert gender to numeric (0 for Male, 1 for Female)
gender_df['gender'] = (gender_df['gender'] == 'Female').astype(int)

all_pvals_gender = []
emotion_order_gender = []
coefficients_gender = [] 
sample_sizes_gender = []

for emotion in emotion_columns:
    try:
        results, n = run_ordinal_regression(gender_df, emotion, independent_var='gender')
        all_pvals_gender.append(results.pvalues['gender'])
        emotion_order_gender.append(emotion)
        coefficients_gender.append(results.params['gender'])
        sample_sizes_gender.append(n)
    except Exception as e:
        print(f"Error processing {emotion}: {str(e)}")

# Apply FDR correction
rejected_gender, adjusted_pvals_gender = fdrcorrection(all_pvals_gender, alpha=0.05, method='indep')

# Print all results, sorted by original p-value
print("\nAll gender results sorted by p-value:")
print("\nEmotion | Coefficient | Original P | Adjusted P | Significant after FDR")
print("-" * 85)

# Create sorted indices
sorted_indices = sorted(range(len(all_pvals_gender)), key=lambda k: all_pvals_gender[k])

for i in sorted_indices:
    print(f"{emotion_order_gender[i]} | {coefficients_gender[i]:.4f} | {all_pvals_gender[i]:.4f} | {adjusted_pvals_gender[i]:.4f} | {'Yes' if rejected_gender[i] else 'No'}")

# Print summary of significant results
print("\n" + "="*50)
print("\nSignificant gender relationships after FDR correction (q < 0.05):")
print("\nEmotion | Coefficient | Original P | Adjusted P | n")
print("-" * 70)
significant_count = 0
for i in sorted_indices:
    if rejected_gender[i]:
        significant_count += 1
        print(f"{emotion_order_gender[i]} | {coefficients_gender[i]:.4f} | {all_pvals_gender[i]:.4f} | {adjusted_pvals_gender[i]:.4f} | {sample_sizes_gender[i]}")

if significant_count == 0:
    print("No relationships remained significant after FDR correction.")

print(f"\nNumber of tests conducted: {len(all_pvals_gender)}")
print(f"Number of significant results before correction: {sum(p < 0.05 for p in all_pvals_gender)}")
print(f"Number of significant results after FDR correction: {sum(rejected_gender)}")

# Print significant results before FDR correction (p < 0.05)
print("\n" + "="*50)
print("\nSignificant gender relationships before FDR correction (p < 0.05):")
print("\nEmotion | Coefficient | Direction | P-value | n") 
print("-" * 80)
uncorrected_significant = 0
for i in sorted_indices:
    if all_pvals_gender[i] < 0.05:
        uncorrected_significant += 1
        direction = "Higher in females" if coefficients_gender[i] > 0 else "Higher in males"
        print(f"{emotion_order_gender[i]} | {coefficients_gender[i]:.4f} | {direction} | {all_pvals_gender[i]:.4f} | {sample_sizes_gender[i]}")

if uncorrected_significant == 0:
    print("No relationships were significant before FDR correction.")


Optimization terminated successfully.
         Current function value: 1.067325
         Iterations: 98
         Function evaluations: 175
Optimization terminated successfully.
         Current function value: 1.163178
         Iterations: 78
         Function evaluations: 135
Optimization terminated successfully.
         Current function value: 1.088406
         Iterations: 55
         Function evaluations: 100
Optimization terminated successfully.
         Current function value: 1.127718
         Iterations: 124
         Function evaluations: 218
Optimization terminated successfully.
         Current function value: 1.069878
         Iterations: 90
         Function evaluations: 148
Optimization terminated successfully.
         Current function value: 1.281642
         Iterations: 125
         Function evaluations: 210
Optimization terminated successfully.
         Current function value: 1.156476
         Iterations: 113
         Function evaluations: 199
Optimization terminated 